In [ ]:
import os
import getpass
import requests
from langchain.chat_models import init_chat_model
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.agents import Tool



In [ ]:
openai_api_key = getpass.getpass("لطفاً API Key مدل OpenAI را وارد کنید: ")
twelve_api_key = getpass.getpass("لطفاً API Key Twelve Data را وارد کنید: ")

os.environ["OPENAI_API_KEY"] = openai_api_key



In [ ]:
model = init_chat_model("gpt-4o-mini", model_provider="openai")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
- از مثال‌ها و توضیحات قابل فهم استفاده کن.

سوال کاربر: {user_question}
"""

prompt = PromptTemplate(
    input_variables=["user_question"],
    template=prompt_template
)


In [ ]:
def fetch_fundamental_and_news(symbol: str) -> str:
    base_url = "https://api.twelvedata.com"
    
    fundamental_url = f"{base_url}/fundamental?symbol={symbol}&apikey={twelve_api_key}"
    news_url = f"{base_url}/news?symbol={symbol}&apikey={twelve_api_key}"

    try:
        fundamental_res = requests.get(fundamental_url)
        fundamental_data = fundamental_res.json()

        news_res = requests.get(news_url)
        news_data = news_res.json()

        summary = "اطلاعات بنیادی:\n"
        if "fundamental" in fundamental_data:
            fund = fundamental_data["fundamental"]
            summary += f"قیمت فعلی: {fund.get('price', 'ناموجود')}\n"
            summary += f"EPS: {fund.get('eps', 'ناموجود')}\n"
            summary += f"P/E: {fund.get('pe', 'ناموجود')}\n"
            summary += f"درآمد: {fund.get('revenue', 'ناموجود')}\n"
        else:
            summary += "داده بنیادی موجود نیست.\n"

        summary += "\nاخبار اخیر:\n"
        if "data" in news_data:
            for i, item in enumerate(news_data["data"][:3]):
                summary += f"{i+1}. {item['title']} - {item['datetime']}\n"
        else:
            summary += "اخباری یافت نشد.\n"

        return summary
    except Exception as e:
        return f"خطا در دریافت داده‌ها: {e}"



In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
شما یک مشاور اقتصادی حرفه‌ای هستید. هدف شما این است که به سوالات اقتصادی، مالی و سرمایه‌گذاری پاسخ دهید.
- پاسخ‌ها باید **فارسی روان و قابل فهم** باشند.
- اگر سوال غیر اقتصادی باشد، به صورت مودبانه اطلاع بده که تخصص شما فقط در زمینه اقتصادی است.
- تحلیل دقیق، راهکار عملی و مشاوره کاربردی ارائه بده.
- از مثال‌ها و توضیحات قابل فهم استفاده کن.

اطلاعات بنیادی و اخبار مرتبط با نماد {symbol}:
{fundamental_and_news}

سوال کاربر: {user_question}
"""

prompt = PromptTemplate(
    input_variables=["symbol", "fundamental_and_news", "user_question"],
    template=prompt_template
)


In [ ]:
from langchain.agents import Tool

def fundamental_search_tool(symbol: str) -> str:
    return fetch_fundamental_and_news(symbol)

tools = [
    Tool(
        name="FundamentalSearch",
        func=fundamental_search_tool,
        description="دریافت اطلاعات بنیادی و اخبار مرتبط با نماد"
    )
]

class EconomicAgent:
    def __init__(self, model, tools, memory):
        self.model = model
        self.tools = {tool.name: tool.func for tool in tools}
        self.memory = memory

    def run(self):
        symbol = input("لطفاً نماد سهم را وارد کنید (مثلا AAPL): ").strip().upper()
        user_question = input("سوال اقتصادی خود را وارد کنید: ").strip()

        fundamental_and_news = self.tools["FundamentalSearch"](symbol)

        final_prompt = prompt.format(
            symbol=symbol,
            fundamental_and_news=fundamental_and_news,
            user_question=user_question
        )

        response = self.model.chat(final_prompt, memory=self.memory)
        return response.message.content




> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "سلام! چطور می توانم به شما کمک کنم؟"
}
```

> Finished chain.

=== پاسخ مشاور اقتصادی ===
سلام! چطور می توانم به شما کمک کنم؟


In [ ]:
agent = EconomicAgent(model=model, tools=tools, memory=memory)
answer = agent.run()

print("\n=== پاسخ مشاور اقتصادی با در نظر گرفتن داده‌های بنیادی و اخبار ===")
print(answer)
